In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# ==========================================
# 1. CẤU HÌNH & KHỞI TẠO
# ==========================================
NUM_CUSTOMERS = 400000  # Số lượng khách hàng unique
YEARS = [2018, 2019, 2020, 2021, 2022, 2023, 2024]
np.random.seed(42)

gender_array = np.random.choice(['M', 'F', 'O'], NUM_CUSTOMERS, p=[0.49, 0.49, 0.02])

print("1. Đang tạo hồ sơ tĩnh (Static Profile)...")
# --- 1.1 Tạo Tuổi & Trình độ ---
age_array = np.random.normal(35, 10, NUM_CUSTOMERS).astype(int)

cond_edu = [age_array < 22, age_array <= 25]
choice_edu = [
    np.random.choice([1, 2], NUM_CUSTOMERS, p=[0.9, 0.1]),       # < 22
    np.random.choice([1, 2, 3], NUM_CUSTOMERS, p=[0.3, 0.6, 0.1]) # 22-25
]
default_edu = np.random.choice([1, 2, 3, 4], NUM_CUSTOMERS, p=[0.3, 0.5, 0.15, 0.05])
edu_array = np.select(cond_edu, choice_edu, default=default_edu)

# Xác suất kết hôn cơ bản theo tuổi
base_prob = np.clip((age_array - 22) / 30, 0.05, 0.95)

# Nữ kết hôn sớm hơn chút
gender_adj = np.where(gender_array == 'F', 0.05, 0)

# Xác suất cuối
prob_married = np.clip(base_prob + gender_adj, 0.05, 0.98)

marital_status = np.where(
    np.random.rand(NUM_CUSTOMERS) < prob_married,
    'Married',
    'Single'
)

# --- 1.2 Tạo Tài sản gốc (Initial AUM) ---
log_aum = (
    14 #exp(14) ≈ 1.2 triệu VND, mức tài sản nền 
    + 0.03 * age_array
    + 0.4  * (edu_array >= 3)
    + 0.5  * (marital_status == 'Married')
    + np.random.normal(0, 1.2, NUM_CUSTOMERS)
)

base_aum_init = np.exp(log_aum).astype(np.int64) #Lognormal là phân phối chuẩn cho wealth. Phân phối lệch phải, nhiều người nghèo, ít người giàu 

# --- [PATCH 4] Logic Sở hữu nhà theo Tuổi ---
# Tuổi càng cao -> Xác suất có nhà càng lớn
prob_house = (age_array - 18) / 50 
prob_house = np.clip(prob_house, 0.05, 0.95)
house_ownership = (np.random.rand(NUM_CUSTOMERS) < prob_house).astype(int)

# --- 1.3 Đóng gói DataFrame ---
customers = pd.DataFrame({
    'SOCIF': np.arange(1000000, 1000000 + NUM_CUSTOMERS),
    'C_GIOITINH': gender_array,
    
    'BASE_AGE': age_array,
    'TRINHDO': edu_array,
    'INITIAL_AUM': base_aum_init,
    
    'TTHONNHAN': marital_status,
    'SOHUUNHA': house_ownership, # <--- Đã dùng biến mới
    'NHANVIENBIDV': np.random.choice([0, 1], NUM_CUSTOMERS, p=[0.99, 0.01]),
    'INHERENT_RISK_SCORE': np.random.normal(0, 1, NUM_CUSTOMERS) #Phần rủi ro KHÔNG giải thích được. Để tránh model quá đẹp.
})

# ==========================================
# 2. MỞ RỘNG DỮ LIỆU THEO NĂM (RANDOM WALK)
# ==========================================
print("2. Expand dữ liệu theo năm (Có biến động tài sản)...")

# --- BƯỚC 2.1: TẠO MA TRẬN TĂNG TRƯỞNG ---
n_years = len(YEARS)
growth_matrix = np.random.normal(1.08, 0.15, (NUM_CUSTOMERS, n_years)) #growth_matrix[i, j] = hệ số tăng trưởng AUM của khách i trong năm j
growth_matrix[:, 2] -= 0.15 # 2020, năm covid, tất cả khách bị -15% growth
growth_matrix[:, 3] -= 0.05 # 2021
cum_growth_matrix = np.cumprod(growth_matrix, axis=1) # cumulative product, nhân dồn theo năm

# --- BƯỚC 2.2: GÁN VÀO DATAFRAME ---
df_list = []

for i, year in enumerate(YEARS):
    temp_df = customers.copy()
    temp_df['year'] = year

    # AUM động
    current_growth_factors = cum_growth_matrix[:, i]
    temp_df['BASE_AUM'] = (
        temp_df['INITIAL_AUM'] * current_growth_factors
    ).astype(np.int64).clip(0)

    # Segment theo AUM
    cond_segment = [
        temp_df['BASE_AUM'] < 100_000_000,
        temp_df['BASE_AUM'] < 1_000_000_000
    ]
    choice_segment = ['Mass', 'Upper']
    temp_df['final_CST_MKT_SEG'] = np.select(
        cond_segment, choice_segment, default='Private'
    )

    df_list.append(temp_df)

# ===============================
# 4. GỘP & HẬU XỬ LÝ
# ===============================
df = pd.concat(df_list, ignore_index=True)
df = df.sort_values(by=['SOCIF', 'year'])
df['TUOI'] = df['BASE_AGE'] + (df['year'] - 2018)
df.drop(columns=['INITIAL_AUM'], inplace=True)

# ==========================================
# 4. SINH BIẾN TÀI CHÍNH (RE-ORDERED & PATCHED)
# ==========================================
print("4. Sinh biến tài chính (Logic AUM -> INCOME -> CBAL)...")
N = len(df)
risk_factor = df['INHERENT_RISK_SCORE']

# --- [PATCH 2] BƯỚC 4.1: SINH INCOME TRƯỚC (Dựa trên AUM & Trình độ) ---
# Logic: Có tài sản thì mới sinh ra thu nhập
base_income_from_aum = np.log1p(df['BASE_AUM']) * 1_200_000 
edu_multiplier = df['TRINHDO'].map({1: 0.8, 2: 1.0, 3: 1.5, 4: 2.5})

df['INCOME'] = (base_income_from_aum * edu_multiplier * np.random.uniform(0.8, 1.5, N)).astype(int)
df['INCOME'] = df['INCOME'].clip(5_000_000, 500_000_000)

# --- BƯỚC 4.2: SINH DƯ NỢ (CBAL) DỰA TRÊN INCOME ---
# Logic: Nhu cầu vay = Khẩu vị rủi ro (Random) * Khả năng trả nợ (Income)
base_loan_propensity = np.random.lognormal(17.5, 0.8, N)  # khẩu vị vay vốn. Dùng log normal vì đa số vay ít, một số vay nhiều 
income_scale = df['INCOME'] / df['INCOME'].median() # khẩ năng trả nợ 

# Người risk cao vay nhiều hơn, người thu nhập cao vay nhiều hơn
risk_adj = np.clip(1 - risk_factor * 0.1, 0.5, 1.5) # điều chỉnh theo rủi ro 
df['CBAL'] = (base_loan_propensity * income_scale * risk_adj).astype(int)

# Zero Balance Logic
mask_zero_debt = (df['INCOME'] < df['INCOME'].median()) & (np.random.rand(N) < 0.25) # 25% thu nhập thấp, không vay. Tránh dataset ai cũng nợ 
df.loc[mask_zero_debt, 'CBAL'] = 0

# CBALORG 
df['CBALORG'] = (df['CBAL'] * np.random.uniform(1.0, 1.2, N)).astype(int)

df.loc[df['CBAL'] == 0, 'CBALORG'] = (
    np.random.lognormal(13, 0.6, (df['CBAL'] == 0).sum()).astype(int)
)

# --- [PATCH 3] Logic Toán học Max/Min/Avg ---
df['CBAL_AVG'] = (df['CBAL'] * np.random.uniform(0.8, 1.2, N)).astype(int)

temp_max = (df['CBAL_AVG'] * np.random.uniform(1.1, 1.5, N)).astype(int)
# Fix lỗi Max < Current bằng np.maximum.reduce
df['CBAL_MAX'] = np.maximum.reduce([temp_max, df['CBAL'], df['CBAL_AVG']]) 

temp_min = (df['CBAL_AVG'] * np.random.uniform(0.5, 0.9, N)).astype(int)
# Fix lỗi Min > Current bằng np.minimum.reduce
df['CBAL_MIN'] = np.minimum.reduce([temp_min, df['CBAL'], df['CBAL_AVG']]) 

# Hạn mức
limit_basis = np.maximum(df['CBALORG'], df['INCOME'] * 5) #không cấp hạn mức thấp hơn dư nợ đã từng cấp, và cũng không vượt quá khả năng trả nợ theo income.
df['AFLIMT_MAX'] = (limit_basis * np.random.uniform(1.0, 2.0, N)).astype(int)
df['AFLIMT_MIN'] = (df['AFLIMT_MAX'] * 0.9).astype(int)
df['AFLIMT_AVG'] = (df['AFLIMT_MAX'] + df['AFLIMT_MIN']) // 2
# df['AFLIMT_SUM'] = df['AFLIMT_MAX']

# --- BƯỚC 4.3: SINH CÁC BIẾN CÒN LẠI (LTV, DEPOSIT) ---
# LTV (Dùng Income và CBAL đã đồng bộ
collateral_value = (
    df['BASE_AUM'] * np.random.uniform(0.5, 1.5, N) #
    + df['SOHUUNHA'] * np.random.uniform(1e9, 3e9, N) #nhà là tài sản đảm bảo 
)

df['COLLATERAL_VALUE'] = collateral_value.astype(int)

df['LTV'] = (df['CBAL'] / (df['COLLATERAL_VALUE'] + 1)) * 100
df['LTV'] = df['LTV'].clip(0, 150)


# [PATCH 1] Tiền gửi (DEPOSIT) tương quan với AUM
deposit_ratio = np.random.uniform(0.1, 0.9, N)
df['N_AVG_DEPOSIT_12M'] = (df['BASE_AUM'] * deposit_ratio).astype(int)

# Ép phân khúc Bad
# df['N_AVG_DEPOSIT_3M'] = (df['N_AVG_DEPOSIT_12M'] * np.random.uniform(0.8, 1.2, N)).astype(int)
# df['N_AVG_DEPOSIT_6M'] = (df['N_AVG_DEPOSIT_12M'] * np.random.uniform(0.9, 1.1, N)).astype(int)
# df['N_AVG_DEPOSIT_9M'] = (df['N_AVG_DEPOSIT_12M'] * np.random.uniform(0.95, 1.05, N)).astype(int)

df['N_AVG_DD_12M'] = (df['N_AVG_DEPOSIT_12M'] * 0.3).astype(int) # biến CASA, tiền gửi không kỳ hạn 
df['N_AVG_CD_12M'] = df['N_AVG_DEPOSIT_12M'] - df['N_AVG_DD_12M'] # biến tiền gửi kỳ hạn 
# threshold = df['N_AVG_DEPOSIT_12M'].median()
# df['FLAG_DEPOSIT'] = (df['N_AVG_DEPOSIT_12M'] >= threshold).astype(int)
df['FLAG_SALARY_ACC'] = np.random.choice([0, 1], N)

# --- Các biến thời hạn & Lãi suất ---
df['DURATION_MAX'] = np.random.choice([12, 24, 36, 60], N)
# df['DURATION_AVG'] = df['DURATION_MAX']
df['REMAINING_DURATION_MAX'] = (df['DURATION_MAX'] * np.random.uniform(0.1, 0.9, N)).astype(int)      
df['TIME_TO_OP_MAX'] = df['DURATION_MAX'] - df['REMAINING_DURATION_MAX']
df['RATE_AVG'] = np.random.normal(9.5, 2.0, N).clip(5, 20)

# ==========================================
# 5. BIẾN HÀNH VI & NỢ XẤU CHI TIẾT
# ==========================================
print("5. Mapping chi tiết nợ quá hạn...")

# DPD base từ risk + macro
base_dpd = (
    np.maximum(0, np.random.normal(15, 20, N))
    + (df['INHERENT_RISK_SCORE'] > 1) * np.random.randint(30, 90, N)
    + (df['year'].isin([2020, 2021])) * np.random.randint(10, 40, N)
)

df['MAX_DPD_12M'] = base_dpd.clip(0, 360).astype(int)
df['MAX_DPD_12M_OBS'] = (
    df['MAX_DPD_12M'] * np.random.uniform(0.8, 1.0, N)
).astype(int)

def cic_from_dpd(dpd):
    if dpd >= 180: return np.random.choice([4, 5])
    if dpd >= 90:  return 3
    if dpd >= 30:  return 2
    return 1

df['MAX_NHOMNOCIC'] = df['MAX_DPD_12M'].apply(cic_from_dpd)
df['AVG_OD_DPD_12M'] = (
    df['MAX_DPD_12M_OBS'] * np.random.uniform(0.3, 0.7, N)
).astype(int)

extra_od = np.random.poisson(lam=2, size=N) * np.random.uniform(5, 20, N) #các lần trễ khác
df['SUM_ALL_OD_12M'] = df['MAX_DPD_12M_OBS'] + extra_od

# df['MAX_AFCPNO_OD_12M'] = df['MAX_DPD_12M_OBS'] 
# df['MAX_NHOMNOCUOI'] = df['MAX_NHOMNOCIC']
df['XULYNO'] = np.where(
    (df['MAX_DPD_12M_OBS'] >= 90) & (np.random.rand(N) < 0.2),
    1, 0
) # 20% khách quá hạn nợ max trên 90 ngày bị xử lý nợ 

df['N_AVG_OVERDUE_CBAL_12M'] = np.where(df['MAX_DPD_12M_OBS'] > 0, df['CBAL'] * np.random.uniform(0.1, 1.0, N), 0).astype(int) #Dư nợ trung bình bị quá hạn trong 12 tháng gần nhất. Nếu có phát sinh quá hạn, thì mới có dư nợ quá hạn. Có khách chỉ một phần dư nợ bị quá hạn, có khách gần như toàn bộ dư nợ bị quá hạn
# df['N_MAX_OVERDUE_CBAL_12M'] = df['N_AVG_OVERDUE_CBAL_12M']

df['BAD'] = np.where(df['MAX_DPD_12M_OBS'] >= 90, 1, 0)
df = df.sort_values(['SOCIF', 'year'])
df['BAD_NEXT_12M'] = df.groupby('SOCIF')['BAD'].shift(-1)

# Penalize deposit sau khi đã có DPD (NO LEAK)
df.loc[df['MAX_DPD_12M_OBS'] >= 60, 'N_AVG_DEPOSIT_12M'] *= 0.5

mask_penalized = df['MAX_DPD_12M_OBS'] >= 60

df.loc[mask_penalized, 'N_AVG_DD_12M'] = (
    df.loc[mask_penalized, 'N_AVG_DEPOSIT_12M'] * 0.3
).astype(int)

df.loc[mask_penalized, 'N_AVG_CD_12M'] = (
    df.loc[mask_penalized, 'N_AVG_DEPOSIT_12M']
    - df.loc[mask_penalized, 'N_AVG_DD_12M']
)

# df['FLAG_DEPOSIT'] = np.where(df['N_AVG_DEPOSIT_12M'] > 500_000, 1, 0)

# Re-calc các window khác cho consistency
df['N_AVG_DEPOSIT_3M'] = (df['N_AVG_DEPOSIT_12M'] * np.random.uniform(0.8, 1.2, N)).astype(int)
df['N_AVG_DEPOSIT_6M'] = (df['N_AVG_DEPOSIT_12M'] * np.random.uniform(0.9, 1.1, N)).astype(int)
df['N_AVG_DEPOSIT_9M'] = (df['N_AVG_DEPOSIT_12M'] * np.random.uniform(0.95, 1.05, N)).astype(int)
threshold = df['N_AVG_DEPOSIT_12M'].median()
df['FLAG_DEPOSIT'] = (df['N_AVG_DEPOSIT_12M'] >= threshold).astype(int)
    
# ==========================================
# 6. BIẾN VĨ MÔ (MACRO)
# ==========================================
print("6. Ghép nối dữ liệu vĩ mô...")
macro_data = {
    2018: {'GDP': 7.08, 'CPI': 3.54, 'UR': 2.19, 'IIP': 10},
    2019: {'GDP': 7.02, 'CPI': 2.79, 'UR': 2.17, 'IIP': 9},
    2020: {'GDP': 2.91, 'CPI': 3.23, 'UR': 2.48, 'IIP': 3},
    2021: {'GDP': 2.58, 'CPI': 1.84, 'UR': 3.22, 'IIP': 4},
    2022: {'GDP': 8.02, 'CPI': 3.15, 'UR': 2.32, 'IIP': 8},
    2023: {'GDP': 5.05, 'CPI': 3.25, 'UR': 2.28, 'IIP': 5},
    2024: {'GDP': 6.00, 'CPI': 3.00, 'UR': 2.30, 'IIP': 6},
}
def get_macro(year, ind): 
    return macro_data.get(year, {}).get(ind, 0)

for ind in ['GDP', 'CPI', 'UR', 'IIP']:
    col = f'REAL_{ind}' if ind == 'GDP' else ind
    df[col] = df['year'].apply(lambda y: get_macro(y, ind))
    df[f'{col}_GROWTH_12M'] = df[col] * np.random.uniform(0.9, 1.1, N) #đưa thêm nhiễu cá nhân để phản ánh mức độ khách hàng chịu tác động khác nhau từ cùng một điều kiện kinh tế vĩ mô

# ==========================================
# 7. SINH CHI TIẾT CẤU TRÚC NỢ
# ==========================================
print("7. Đang sinh chi tiết cấu trúc nợ...")

# Cấu trúc kỳ hạn. Phân rã CBAL theo kỳ hạn (Short / Mid / Long)
w_short = np.random.uniform(0, 1, N)
w_mid = np.random.uniform(0, 1, N)
w_long = np.random.uniform(0, 1, N)
total_w = w_short + w_mid + w_long
w_short /= total_w
w_mid /= total_w
w_long /= total_w

df['CBAL_SHORTTERM_LOAN'] = (df['CBAL'] * w_short).astype(int)
df['CBAL_MIDTERM_LOAN'] = (df['CBAL'] * w_mid).astype(int)
df['CBAL_LONGTERM_LOAN'] = df['CBAL'] - df['CBAL_SHORTTERM_LOAN'] - df['CBAL_MIDTERM_LOAN']

df.loc[df['CBAL'] == 0, ['CBAL_SHORTTERM_LOAN', 'CBAL_MIDTERM_LOAN', 'CBAL_LONGTERM_LOAN']] = 0
df['HAS_SHORTTERM_LOAN'] = np.where(df['CBAL_SHORTTERM_LOAN'] > 0, 1, 0)
df['HAS_MIDTERM_LOAN'] = np.where(df['CBAL_MIDTERM_LOAN'] > 0, 1, 0)
df['HAS_LONGTERM_LOAN'] = np.where(df['CBAL_LONGTERM_LOAN'] > 0, 1, 0)

df.loc[df['CBAL'] == 0, ['N_AVG_OVERDUE_CBAL_12M']] = 0

# Biến động LTV
df['MAX_LTV_MO'] = df['LTV'] * np.random.uniform(1.0, 1.1, N)
df['MIN_LTV_MO'] = df['LTV'] * np.random.uniform(0.8, 1.0, N)
df['AVG_LTV_MO'] = (df['MAX_LTV_MO'] + df['MIN_LTV_MO']) / 2

# Tỷ lệ DTI
df['CBAL_TO_INC_12MON'] = df['CBAL'] / (df['INCOME'] + 1)
df['CBAL_TO_INC_12MON'] = df['CBAL_TO_INC_12MON'].clip(0, 5)
df['CBAL_TO_INC_9MON'] = df['CBAL_TO_INC_12MON'] * np.random.uniform(0.95, 1.05, N)
df['CBAL_TO_INC_6MON'] = df['CBAL_TO_INC_12MON'] * np.random.uniform(0.9, 1.1, N)
df['CBAL_TO_INC_3MON'] = df['CBAL_TO_INC_12MON'] * np.random.uniform(0.85, 1.15, N)

# Thông tin trả nợ (Dùng CBAL_AVG để tính lãi)
df['INTEREST_12M'] = (df['CBAL_AVG'] * (df['RATE_AVG'] / 100)).astype(int) #dư nợ bình quân * (lãi suất bình quân năm / 100) = Tiền lãi ước tính phát sinh trong 12 tháng
df['INTEREST'] = (df['INTEREST_12M'] / 12).astype(int) #% 12 tháng chia 12 

df['PRINPICAL_PYMT_FRQ_ID_MAX'] = np.random.choice([1, 1, 1, 3, 6], N, p=[0.7, 0.1, 0.1, 0.05, 0.05])
df['N_PAYMENT_GOC_LAI'] = np.where(
    df['CBAL'] > 0,
    np.clip((12 / df['PRINPICAL_PYMT_FRQ_ID_MAX']).astype(int), 1, 12),
    0
)

# df['N_PAYMENT_GOC'] = np.where(df['CBAL'] > 0, np.random.randint(1, 13, N), 0)
# df['N_PAYMENT_LAI'] = np.where(df['CBAL'] > 0, np.random.randint(1, 13, N), 0)

# df['INT_PYMT_FRQ_ID_MAX'] = df['PRINPICAL_PYMT_FRQ_ID_MAX']

# Codes
# Mục đích cho khoản lớn nhất (thiên về vay có TSĐB)
# 1–3: tiêu dùng
# 4–5: thẻ / thấu chi
# 6–7: vay TSĐB nhỏ
# 8–9: khác
df['PURCOD_MAX'] = np.random.choice(
    [1,2,3,4,5,6,7,8,9],
    N,
    p=[0.05,0.05,0.1,0.15,0.15,0.25,0.15,0.05,0.05]
)

# Mục đích cho khoản nhỏ nhất (thiên về tiêu dùng / thẻ)
df['PURCOD_MIN'] = np.random.choice( 
    [1,2,3,4,5,6,7,8,9],
    N,
    p=[0.2,0.2,0.2,0.15,0.15,0.05,0.03,0.01,0.01]
)

# df['CFORGD'] = np.random.choice(['C', 'D', 'O', np.nan], N, p=[0.4, 0.3, 0.1, 0.2]) 

# CBALORG Variation
df['CBALORG_MAX'] = df['CBALORG']
df['CBALORG_MIN'] = (df['CBALORG'] * 0.9).astype(int)
df['CBALORG_AVG'] = (df['CBALORG_MAX'] + df['CBALORG_MIN']) // 2

# ==========================================
# 8. GÁN NHÃN & TẠO DIRTY 
# ==========================================
def assign_sample(year):
    if year <= 2021: return 'TRAIN'
    if year == 2022: return 'OOS'
    if year >= 2023: return 'OOT'
    return None

df = df[df['year'] <= 2023].copy()

df['SAMPLE_TYPE'] = df['year'].apply(assign_sample)

# Inject Dirty Data
df.loc[np.random.choice(df.index, 50), 'TUOI'] = 14 
df.loc[np.random.choice(df.index, 30), 'LTV'] = 500
df.loc[np.random.choice(df.index, 20), 'DURATION_MAX'] = -12

print("-" * 30)
print(f"XONG! Kích thước: {df.shape}")
print(f"Số cột: {len(df.columns)}")
print("Phân phối Sample:")
print(df['SAMPLE_TYPE'].value_counts())
print(f"\nSố lượng KH Zero Balance: {(df['CBAL'] == 0).sum()}")

1. Đang tạo hồ sơ tĩnh (Static Profile)...
2. Expand dữ liệu theo năm (Có biến động tài sản)...
4. Sinh biến tài chính (Logic AUM -> INCOME -> CBAL)...
5. Mapping chi tiết nợ quá hạn...


C:\Users\PC\AppData\Local\Temp\ipykernel_10296\2442043399.py:253: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[4793655.  1961408.5  307169.5 ... 2322971.5 2552899.  7016822.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['MAX_DPD_12M_OBS'] >= 60, 'N_AVG_DEPOSIT_12M'] *= 0.5
C:\Users\PC\AppData\Local\Temp\ipykernel_10296\2442043399.py:261: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[3355559.  1372986.5  215019.5 ... 1626080.5 1787030.  4911776.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[mask_penalized, 'N_AVG_CD_12M'] = (


6. Ghép nối dữ liệu vĩ mô...
7. Đang sinh chi tiết cấu trúc nợ...
------------------------------
XONG! Kích thước: (2400000, 75)
Số cột: 75
Phân phối Sample:
SAMPLE_TYPE
TRAIN    1600000
OOS       400000
OOT       400000
Name: count, dtype: int64

Số lượng KH Zero Balance: 301119


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df

,SOCIF,C_GIOITINH,BASE_AGE,TRINHDO,TTHONNHAN,SOHUUNHA,NHANVIENBIDV,INHERENT_RISK_SCORE,year,BASE_AUM,final_CST_MKT_SEG,TUOI,INCOME,CBAL,CBALORG,CBAL_AVG,CBAL_MAX,CBAL_MIN,AFLIMT_MAX,AFLIMT_MIN,AFLIMT_AVG,COLLATERAL_VALUE,LTV,N_AVG_DEPOSIT_12M,N_AVG_DD_12M,N_AVG_CD_12M,FLAG_SALARY_ACC,DURATION_MAX,REMAINING_DURATION_MAX,TIME_TO_OP_MAX,RATE_AVG,MAX_DPD_12M,MAX_DPD_12M_OBS,MAX_NHOMNOCIC,AVG_OD_DPD_12M,SUM_ALL_OD_12M,XULYNO,N_AVG_OVERDUE_CBAL_12M,BAD,BAD_NEXT_12M,N_AVG_DEPOSIT_3M,N_AVG_DEPOSIT_6M,N_AVG_DEPOSIT_9M,FLAG_DEPOSIT,REAL_GDP,REAL_GDP_GROWTH_12M,CPI,CPI_GROWTH_12M,UR,UR_GROWTH_12M,IIP,IIP_GROWTH_12M,CBAL_SHORTTERM_LOAN,CBAL_MIDTERM_LOAN,CBAL_LONGTERM_LOAN,HAS_SHORTTERM_LOAN,HAS_MIDTERM_LOAN,HAS_LONGTERM_LOAN,MAX_LTV_MO,MIN_LTV_MO,AVG_LTV_MO,CBAL_TO_INC_12MON,CBAL_TO_INC_9MON,CBAL_TO_INC_6MON,CBAL_TO_INC_3MON,INTEREST_12M,INTEREST,PRINPICAL_PYMT_FRQ_ID_MAX,N_PAYMENT_GOC_LAI,PURCOD_MAX,PURCOD_MIN,CBALORG_MAX,CBALORG_MIN,CBALORG_AVG,SAMPLE_TYPE
0,1000000,M,31,1,Single,0,0,-0.187304,2018,8350987,Mass,31,15100579,21158353,21681716,20056369,24823783,12521561,110602817,99542535,105072676,12205813,150.000000,3359817.0,1007945,2351872.0,0,36,22,14,12.031186,0,0,1,0,26.321614,0,0,0,0.0,3452415,3251488,3486653,1,7.08,6.862700,3.54,3.627965,2.19,2.316208,10,9.885668,7790094,406606,12961653,1,1,1,164.772957,130.664924,147.718941,1.401162,1.349464,1.323832,1.227912,2413019,201084,1,12,7,2,21681716,19513544,20597630,TRAIN
400000,1000000,M,31,1,Single,0,0,-0.187304,2019,7108901,Mass,32,19442835,13340870,15961640,12172701,14995934,10510934,119421159,107479043,113450101,5312785,150.000000,2729371.0,818811,1910560.0,0,12,4,8,11.397522,31,29,2,15,94.968445,0,11872364,0,0.0,2272365,2747597,2679189,1,7.02,7.208454,2.79,2.949272,2.17,2.377729,9,8.584035,6316029,3619361,3405480,1,1,1,156.067121,145.699321,150.883221,0.686159,0.673632,0.717038,0.644344,1387386,115615,1,12,7,4,15961640,14365476,15163558,TRAIN
800000,1000000,M,31,1,Single,0,0,-0.187304,2020,7176536,Mass,33,13778504,4548646,4619557,5430470,7849903,4548646,129053492,116148142,122600817,7376905,61.660620,4002591.0,1200777,2801814.0,1,60,27,33,11.357797,36,30,2,9,89.373708,0,4214873,0,0.0,3464344,3752476,3974660,1,2.91,2.659522,3.23,3.544882,2.48,2.690635,3,2.808829,1971794,1837571,739281,1,1,1,63.893815,51.945418,57.919617,0.330126,0.342328,0.316576,0.288590,616781,51398,1,12,7,3,4619557,4157601,4388579,TRAIN
1200000,1000000,M,31,1,Single,0,0,-0.187304,2021,9272487,Mass,34,12411213,0,103707,0,0,0,80318868,72286981,76302924,9319635,0.000000,7428733.0,2228619,5200114.0,1,12,6,6,7.261593,54,52,2,20,63.248132,0,0,0,0.0,8457075,7233091,7180595,1,2.58,2.539305,1.84,1.821581,3.22,3.159800,4,4.253249,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,0,6,2,103707,93336,98521,TRAIN
1600000,1000000,M,31,1,Single,0,0,-0.187304,2022,9876502,Mass,35,15696730,12191819,13060761,13842193,18003226,11750395,127215295,114493765,120854530,8950153,136.219097,6909583.0,2072874,4836709.0,0,12,10,2,8.759408,6,5,1,2,17.993844,0,2681822,0,0.0,8178810,6222978,7162117,1,8.02,8.036941,3.15,3.316337,2.32,2.109696,8,7.567788,6286054,581874,5323891,1,1,1,149.502021,134.721919,142.111970,0.776711,0.812736,0.799901,0.790682,1212494,101041,1,12,6,1,13060761,11754684,12407722,OOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799999,1399999,F,33,1,Single,1,0,-0.589531,2019,21325575,Mass,34,17071103,25395157,27889020,23227200,28799638,16482409,105741966,95167769,100454867,2110378732,1.203346,9445522.0,2833656,6611866.0,0,24,16,8,12.287827,34,31,2,17,40.198354,0,14204400,0,0.0,9462194,9558800,9014800,1,7.02,7.413089,2.79,2.717291,2.17,2.074260,9,8.501576,4427376,9307124,11660657,1,1,1,1.317888,1.047448,1.182668,1.487611,1.526786,1.481176,1.

In [5]:
df['INTEREST_12M'].max()

375268184

In [6]:
df.columns

Index(['SOCIF', 'C_GIOITINH', 'BASE_AGE', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK_SCORE', 'year', 'BASE_AUM',
       'final_CST_MKT_SEG', 'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'CBAL_AVG',
       'CBAL_MAX', 'CBAL_MIN', 'AFLIMT_MAX', 'AFLIMT_MIN', 'AFLIMT_AVG',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DD_12M',
       'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'DURATION_MAX',
       'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX', 'RATE_AVG', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'MAX_NHOMNOCIC', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M',
       'XULYNO', 'N_AVG_OVERDUE_CBAL_12M', 'BAD', 'BAD_NEXT_12M',
       'N_AVG_DEPOSIT_3M', 'N_AVG_DEPOSIT_6M', 'N_AVG_DEPOSIT_9M',
       'FLAG_DEPOSIT', 'REAL_GDP', 'REAL_GDP_GROWTH_12M', 'CPI',
       'CPI_GROWTH_12M', 'UR', 'UR_GROWTH_12M', 'IIP', 'IIP_GROWTH_12M',
       'CBAL_SHORTTERM_LOAN', 'CBAL_MIDTERM_LOAN', 'CBAL_LONGTERM_LOAN',
       'HAS_SHORTTERM_LOAN', 'HAS_MIDTERM_LOAN', 'HAS_LONGTERM_LOAN',
      

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
df['INCOME'].min()

7542231

In [9]:
df['INCOME'].max()

96028527

In [10]:
df['CBAL'].value_counts()

CBAL
0            301119
31987918          4
57077831          3
20724360          3
40949277          3
              ...  
70396148          1
81609556          1
187496775         1
75542403          1
25403628          1
Name: count, Length: 2078707, dtype: int64

In [11]:
df.to_csv('gen_data.csv', index=False)

In [12]:
# df_train = df[df['SAMPLE_TYPE'] == 'TRAIN']
# df_OOS = df[df['SAMPLE_TYPE'] == 'OOS']
# df_OOT = df[df['SAMPLE_TYPE'] == 'OOT']

In [13]:
# df_train.drop(['SAMPLE_TYPE'], axis=1, inplace=True)
# df_train.shape

In [14]:
# df_OOS.drop(['SAMPLE_TYPE'], axis=1, inplace=True)
# df_OOS.shape 

In [15]:
# df_OOT.drop(['SAMPLE_TYPE'], axis=1, inplace=True)
# df_OOT.shape

In [16]:
# df_train.to_csv('train.csv', index=False)
# df_OOS.to_csv('oos.csv', index=False)
# df_OOT.to_csv('oot.csv', index=False)